In [2]:
import random
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
from pandas import Timestamp
from datetime import datetime
from time import time
from os import getcwd
from os.path import join
%matplotlib inline


# 1. Motivation
-------
The reporting of COVID deaths may be be accurate for the following reason:
- Most of the COVID deaths are associated to other sickness.  It is hard to say it is really casused by COVID.
- Not all the deaths are diagonized for the real cause of the death.
    - Some families of the deaths do not want to do the test.
    - Test requires some additional cost.  This cost is not always justified, esp. in the situation of the shortage of the medical resources when there is serious pandemic or where medical resources is very poor.
    
Another way to determine COVID deaths is to find the excess deaths over the regular deaths in the time before COVID pandemic.   Therefore, we need some kind of data covering a few years before COVID and years in COVID.  The data is available in https://github.com/akarlinsky/world_mortality/blob/main/world_mortality.csv

# 2. Understanding of the data

In [3]:
# https://github.com/akarlinsky/world_mortality/blob/main/world_mortality.csv
# https://github.com/akarlinsky/world_mortality
path = join(getcwd().rstrip('src'), 'data/world_mortality.csv').replace('\\', '/')
DF = pd.read_csv(path)
# DF = pd.read_csv('~/AI/DATA/BigData/DeathBirthRate/world_mortality2015-20220214.csv')
DF.rename(columns = {'country_name':'country'}, inplace=True)
print(DF.head(10))


  iso3c  country  year  time time_unit  deaths
0   ALB  Albania  2015     1   monthly  2490.0
1   ALB  Albania  2015     2   monthly  2139.0
2   ALB  Albania  2015     3   monthly  2051.0
3   ALB  Albania  2015     4   monthly  1906.0
4   ALB  Albania  2015     5   monthly  1709.0
5   ALB  Albania  2015     6   monthly  1561.0
6   ALB  Albania  2015     7   monthly  2008.0
7   ALB  Albania  2015     8   monthly  1687.0
8   ALB  Albania  2015     9   monthly  1569.0
9   ALB  Albania  2015    10   monthly  1560.0


## 2.1 find out how many countries are included

In [4]:
AllCountries = set(DF.country)
print("# of countries included in the data is ", len(AllCountries))

# of countries included in the data is  122


In [5]:
print(DF.year.min(), DF.year.max())

2015 2022


In [6]:
# # of countries included in 2015
len(set(DF[DF.year == 2015].country))

116

In [7]:
# # of countries included in 2015
len(set(DF[DF.year == 2015].country))

116

In [8]:
# # of countries included in 2022
len(set(DF[DF.year == 2022].country))

91

In [9]:
# # of countries included in 2021
len(set(DF[DF.year == 2021].country))

111

In [10]:
# # of countries included in 2020
len(set(DF[DF.year == 2020].country))

122

## 2.2 Find out how many different time_unit
---
We have seen "monthly" time_unit.  Let us find out if there is any other time_unit.

In [11]:
# # of countries included in 2020
set(DF.time_unit)

{'monthly', 'weekly'}

## 2.3 We have seen only two time_unit.  Let us find out if the values are complete for each country

In [12]:
set(DF[DF.year==2020].groupby(['country', 'time_unit']).year.count())
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html
# can distinguish nan and non-nan

{8, 12, 53}

### Since the above return 8, 12 and 53 in the year of 2020.  It is probably that some countries with monthly time_unit do not have complete vaules for all months in 2020.

In [13]:
DF[DF.year==2020].groupby(['country', 'time_unit']).count().head()

,,iso3c,year,time,deaths
country,time_unit,,,,
Albania,monthly,12,12,12,12
Algeria,monthly,12,12,12,12
Andorra,monthly,12,12,12,12
Antigua and Barbuda,monthly,12,12,12,12
Argentina,monthly,12,12,12,12


In [14]:
tmp = DF[DF.year==2020].groupby(['country', 'time_unit']).count()
tmp = tmp.reset_index()
tmp.head()

,country,time_unit,iso3c,year,time,deaths
0,Albania,monthly,12,12,12,12
1,Algeria,monthly,12,12,12,12
2,Andorra,monthly,12,12,12,12
3,Antigua and Barbuda,monthly,12,12,12,12
4,Argentina,monthly,12,12,12,12


In [15]:
# The following shows that there is only one country does not have complete list for the year of 2020
tmp[tmp.year == 8]

,country,time_unit,iso3c,year,time,deaths
34,El Salvador,monthly,8,8,8,8


# 3. Find average annual excess deaths for the years of 2020 and 2021
------
Since 2022 is not finished yet and there is only 90 countries have values in 2022 (even though it is not complete yet), let us focus on the average annual excess deaths for 2020 and 2021.
The simplest way to find the excess deaths is:
1. find the average annual deaths before 2020, i.e. years from 2015 to 2019.  This is the regualr deaths.
- find the average annual deaths for 2020 and 2021. This is the deaths from all causes.
- The difference between the deaths from all causes and the regular deaths is the excess deaths.
- If there is no major event for large scale deaths, then excess deaths should be COVID deaths
- Examples of large scale deaths are big natural disasters like earthquake, tsunami, pandemic, or pandemic and war.  There are no such incidents in 2020 and 2021 other than COVID.   Therefore, this excess deaths must be caused by COVID pandemic.  Ukraine war occurred in 2022.

In [16]:
DF_2015_2019 = DF[DF.year < 2020]
DF_2020_2021 = DF[(DF.year == 2020) | (DF.year==2021)]


In [17]:
Regular = DF_2015_2019.groupby('country').mean().reset_index()
print(Regular.head(30))

                country         year       time         deaths
0               Albania  2017.000000   6.500000    1829.650000
1               Algeria  2018.500000   6.500000   14940.804167
2               Andorra  2017.000000   6.500000      25.850000
3   Antigua and Barbuda  2017.000000   6.500000      47.550000
4             Argentina  2017.000000   6.500000   28449.050000
5               Armenia  2017.000000   6.500000    2253.950000
6                 Aruba  2017.000000   6.500000      59.150000
7             Australia  2016.992337  26.601533    3084.199234
8               Austria  2016.992337  26.601533    1558.122605
9            Azerbaijan  2017.000000   6.500000    4702.566667
10              Bahamas  2017.000000   6.500000     202.900000
11             Barbados  2017.000000   6.500000     220.216667
12              Belarus  2017.000000   6.500000    9994.283333
13              Belgium  2016.992337  26.601533    2099.601533
14               Belize  2017.000000   6.500000     158

In [18]:
DF[(DF.country == 'Australia') & (DF.year < 2020)]

,iso3c,country,year,time,time_unit,deaths
531,AUS,Australia,2015,1,weekly,2925.0
532,AUS,Australia,2015,2,weekly,2772.0
533,AUS,Australia,2015,3,weekly,2770.0
534,AUS,Australia,2015,4,weekly,2768.0
535,AUS,Australia,2015,5,weekly,2680.0
...,...,...,...,...,...,...
787,AUS,Australia,2019,48,weekly,2986.0
788,AUS,Australia,2019,49,weekly,2958.0
789,AUS,Australia,2019,50,weekly,2994.0
790,AUS,Australia,2019,51,weekly,2950.0


In [19]:
DF[(DF.country == 'Australia') & (DF.year < 2022) & (DF.time >= 52)]

,iso3c,country,year,time,time_unit,deaths
582,AUS,Australia,2015,52,weekly,2715.0
583,AUS,Australia,2015,53,weekly,2819.0
635,AUS,Australia,2016,52,weekly,2919.0
687,AUS,Australia,2017,52,weekly,2775.0
739,AUS,Australia,2018,52,weekly,2945.0
791,AUS,Australia,2019,52,weekly,2881.0
843,AUS,Australia,2020,52,weekly,3067.0
844,AUS,Australia,2020,53,weekly,2987.0
896,AUS,Australia,2021,52,weekly,3165.0


### It looks like that:
- when the average of time is 6.5 means that it has complete year from month 1 to month 12 so that the average is 6.5.   The annual average deaths should be 12X of this average.
- when the average of time is 26.60 means that it has complete year from week 1 to week 52 or 53 so that the average is 26.60.  If all year has 52 weeks, this value should be 26.5.  Let us assume there is alway 52 weeks for simplicity.  The annual average deaths should be very close to 52X of this average.

Let us add one column "AverageAnnualUnitCount" to represent the actual number of values in the data for the years we are working on.  The value can help us to know how many values of time_unit we have in the data.   Even though the annual average can be obtained by just the average in the specified time_unit multiplied by 12 or 52 even if some countries do not have complete values for a whole year.


In [20]:
Regular['AverageAnnualUnitCount'] = round(Regular.time * 2 - 1, 0)

In [21]:
Regular.head()

,country,year,time,deaths,AverageAnnualUnitCount
0,Albania,2017.0,6.5,1829.650000,12.0
1,Algeria,2018.5,6.5,14940.804167,12.0
2,Andorra,2017.0,6.5,25.850000,12.0
3,Antigua and Barbuda,2017.0,6.5,47.550000,12.0
4,Argentina,2017.0,6.5,28449.050000,12.0


## The above Regular DataFrame does not have "time_unit"  because it is not numerical and is deleted when we obtain mean or count.  However, it is still better to know what kind of time_unit each country is reporting.   This time_unit can be insert back to Regular by merge method in Pandas.

In [22]:
# obtain time_unit for each country.  We can obtain this value just once by specifying 

import copy as copy 
tmp = copy.copy(DF)
# tmp['time_unit'] = [ 12 for x in tmp.time_unit if x == 'monthly' else 52 ]
time_unit_dict = {'monthly': 12, 'weekly': 52}
tmp['AnnualUnitCount'] = [time_unit_dict[x] for x in tmp.time_unit ]
tmp.head()

,iso3c,country,year,time,time_unit,deaths,AnnualUnitCount
0,ALB,Albania,2015,1,monthly,2490.0,12
1,ALB,Albania,2015,2,monthly,2139.0,12
2,ALB,Albania,2015,3,monthly,2051.0,12
3,ALB,Albania,2015,4,monthly,1906.0,12
4,ALB,Albania,2015,5,monthly,1709.0,12


In [23]:
# We only need one value of AnnualUnitCount for each country.  A quick way to get it is 
tmp = tmp.groupby('country').mean().reset_index()[['country', 'AnnualUnitCount']]
tmp.head()

,country,AnnualUnitCount
0,Albania,12.0
1,Algeria,12.0
2,Andorra,12.0
3,Antigua and Barbuda,12.0
4,Argentina,12.0


In [24]:
Regular.head()

,country,year,time,deaths,AverageAnnualUnitCount
0,Albania,2017.0,6.5,1829.650000,12.0
1,Algeria,2018.5,6.5,14940.804167,12.0
2,Andorra,2017.0,6.5,25.850000,12.0
3,Antigua and Barbuda,2017.0,6.5,47.550000,12.0
4,Argentina,2017.0,6.5,28449.050000,12.0


In [25]:
Regular = pd.merge(Regular, tmp, on='country')
Regular.head()

,country,year,time,deaths,AverageAnnualUnitCount,AnnualUnitCount
0,Albania,2017.0,6.5,1829.650000,12.0,12.0
1,Algeria,2018.5,6.5,14940.804167,12.0,12.0
2,Andorra,2017.0,6.5,25.850000,12.0,12.0
3,Antigua and Barbuda,2017.0,6.5,47.550000,12.0,12.0
4,Argentina,2017.0,6.5,28449.050000,12.0,12.0


In [26]:
# change deaths to annual deaths
Regular['deaths'] = Regular.deaths * Regular.AnnualUnitCount
Regular.head()

,country,year,time,deaths,AverageAnnualUnitCount,AnnualUnitCount
0,Albania,2017.0,6.5,21955.80,12.0,12.0
1,Algeria,2018.5,6.5,179289.65,12.0,12.0
2,Andorra,2017.0,6.5,310.20,12.0,12.0
3,Antigua and Barbuda,2017.0,6.5,570.60,12.0,12.0
4,Argentina,2017.0,6.5,341388.60,12.0,12.0


In [27]:
Regular.columns

Index(['country', 'year', 'time', 'deaths', 'AverageAnnualUnitCount',
       'AnnualUnitCount'],
      dtype='object')

In [28]:
# we don't need everything.  Just select some columns
Regular = Regular[['country', 'deaths', 'AverageAnnualUnitCount']]

In [29]:
Regular.head()

,country,deaths,AverageAnnualUnitCount
0,Albania,21955.80,12.0
1,Algeria,179289.65,12.0
2,Andorra,310.20,12.0
3,Antigua and Barbuda,570.60,12.0
4,Argentina,341388.60,12.0


# 4. Now let us do the same process on DF_2020_2021  for AllCauses

In [30]:
Irregular = DF_2020_2021.groupby('country').mean().reset_index()
Irregular['AverageAnnualUnitCount'] = round(Irregular.time * 2 - 1, 0)
Irregular = pd.merge(Irregular, tmp, on='country')
Irregular['deaths'] = Irregular.deaths * Irregular.AnnualUnitCount
Irregular = Irregular[['country', 'deaths', 'AverageAnnualUnitCount']]
Irregular.head()


,country,deaths,AverageAnnualUnitCount
0,Albania,29092.5,12.0
1,Algeria,235628.0,12.0
2,Andorra,419.0,12.0
3,Antigua and Barbuda,611.5,12.0
4,Argentina,376221.0,12.0


# 5. Combine Regular and AllCauses
### The first deaths and AverageAnnualUnitCount are for Regular, the 2nd is for AllCauses


In [31]:
newDF = pd.concat([Regular, Irregular[['deaths', 'AverageAnnualUnitCount']]], axis=1)
newDF.columns = ['country', 'RegularDeaths', 'RegularAverageAnnualUnitCount', 'IrregularDeaths', 'IrregularAverageAnnualUnitCount']
newDF.head()

,country,RegularDeaths,RegularAverageAnnualUnitCount,IrregularDeaths,IrregularAverageAnnualUnitCount
0,Albania,21955.80,12.0,29092.5,12.0
1,Algeria,179289.65,12.0,235628.0,12.0
2,Andorra,310.20,12.0,419.0,12.0
3,Antigua and Barbuda,570.60,12.0,611.5,12.0
4,Argentina,341388.60,12.0,376221.0,12.0


# 6. Get excess deaths


In [32]:
newDF = pd.concat([newDF, (newDF.IrregularDeaths - newDF.RegularDeaths)], axis=1)
newDF = pd.concat([newDF, (newDF.IrregularDeaths - newDF.RegularDeaths) / newDF.RegularDeaths], axis=1)
newDF.columns = ['country', 'RegularDeaths', 'RegularAverageAnnualUnitCount', 'IrregularDeaths', 'IrregularAverageAnnualUnitCount', 'ExcessDeaths', 'ExcessDeathRate']
newDF.head()

,country,RegularDeaths,RegularAverageAnnualUnitCount,IrregularDeaths,IrregularAverageAnnualUnitCount,ExcessDeaths,ExcessDeathRate
0,Albania,21955.80,12.0,29092.5,12.0,7136.70,0.325049
1,Algeria,179289.65,12.0,235628.0,12.0,56338.35,0.314231
2,Andorra,310.20,12.0,419.0,12.0,108.80,0.350741
3,Antigua and Barbuda,570.60,12.0,611.5,12.0,40.90,0.071679
4,Argentina,341388.60,12.0,376221.0,12.0,34832.40,0.102032


# 7. Get population so that we can calculate deaths per million population from OWID dataset

In [33]:
path = join(getcwd().rstrip('src'),
            'data/owid-covid-data.csv').replace('\\', '/')
data = pd.read_csv(path)

In [34]:
data = data[['location', 'population']]
data.rename(columns = {'location':'country'}, inplace=True)
data.head()


,country,population
0,Afghanistan,40099462.0
1,Afghanistan,40099462.0
2,Afghanistan,40099462.0
3,Afghanistan,40099462.0
4,Afghanistan,40099462.0


In [35]:
data.groupby('country').count().sort_values(by='population', ascending=False)
data.drop_duplicates(subset=['country'], inplace=True)
data.reset_index(drop=True, inplace=True)
print(len(data))
data.head()

244


,country,population
0,Afghanistan,4.009946e+07
1,Africa,1.392394e+09
2,Albania,2.854710e+06
3,Algeria,4.417797e+07
4,Andorra,7.903400e+04


# 8. merge population

In [36]:
con1 = list(data.country)
con2 = list(newDF.country)
pop = []
for c in con2:
    if c in con1:
        pop.append(data[data.country == c].population.values[0])
    else:
        pop.append(0)
pop = pd.Series(pop, name='population')
newDF = pd.concat([newDF, pop], axis=1)
newDF.head()

,country,RegularDeaths,RegularAverageAnnualUnitCount,IrregularDeaths,IrregularAverageAnnualUnitCount,ExcessDeaths,ExcessDeathRate,population
0,Albania,21955.80,12.0,29092.5,12.0,7136.70,0.325049,2854710.0
1,Algeria,179289.65,12.0,235628.0,12.0,56338.35,0.314231,44177969.0
2,Andorra,310.20,12.0,419.0,12.0,108.80,0.350741,79034.0
3,Antigua and Barbuda,570.60,12.0,611.5,12.0,40.90,0.071679,93220.0
4,Argentina,341388.60,12.0,376221.0,12.0,34832.40,0.102032,45276780.0


# 9. calculate deaths per million population and sort by it

In [38]:
rdp = newDF.RegularDeaths / newDF.population
rdp.name = 'RDPM' # RDPM = Regular Deaths Per Million
irdp = newDF.IrregularDeaths / newDF.population
irdp.name = 'IRDPM' # IRDPM = Irregular Deaths Per Million
edp = newDF.ExcessDeaths / newDF.population
edp.name = 'EDPM' # EDPM = Excess Deaths Per Million
newDF = pd.concat([newDF, rdp, irdp, edp], axis=1)
newDF.head()


,country,RegularDeaths,RegularAverageAnnualUnitCount,IrregularDeaths,IrregularAverageAnnualUnitCount,ExcessDeaths,ExcessDeathRate,population,RDPM,IRDPM,EDPM
0,Albania,21955.80,12.0,29092.5,12.0,7136.70,0.325049,2854710.0,0.007691,0.010191,0.002500
1,Algeria,179289.65,12.0,235628.0,12.0,56338.35,0.314231,44177969.0,0.004058,0.005334,0.001275
2,Andorra,310.20,12.0,419.0,12.0,108.80,0.350741,79034.0,0.003925,0.005302,0.001377
3,Antigua and Barbuda,570.60,12.0,611.5,12.0,40.90,0.071679,93220.0,0.006121,0.006560,0.000439
4,Argentina,341388.60,12.0,376221.0,12.0,34832.40,0.102032,45276780.0,0.007540,0.008309,0.000769


In [40]:
newDF.sort_values(by='EDPM', ascending=False, inplace=True)
newDF.reset_index(drop=True, inplace=True)
newDF.columns = ['Country', 'RD', 'RAAUC', 'IRD', 'IRAAUC', 'ED', 'EDR', 'Population', 'RDPM', 'IRDPM', 'EDPM']
newDF.head()
# RD = Regular Deaths
# RAAUC = Regular Average Annual Unit Count
# IRD = Irregular Deaths
# IRAAUC = Irregular Average Annual Unit Count
# ED = Excess Deaths
# EDR = Excess Death Rate
# RDPM = Regular Deaths Per Million
# IRDPM = Irregular Deaths Per Million
# EDPM = Excess Deaths Per Million

,Country,RD,RAAUC,IRD,IRAAUC,ED,EDR,Population,RDPM,IRDPM,EDPM
0,Cabo Verde,2691.200000,12.0,2959.000000,12.0,267.800000,0.099510,0.0,inf,inf,inf
1,French Guiana,890.375479,52.0,1144.495238,53.0,254.119759,0.285407,0.0,inf,inf,inf
2,Réunion,4808.704981,52.0,5467.923810,53.0,659.218829,0.137089,0.0,inf,inf,inf
3,Mayotte,692.735632,52.0,1008.304762,53.0,315.569130,0.455540,0.0,inf,inf,inf
4,Martinique,3271.616858,52.0,4084.228571,53.0,812.611713,0.248382,0.0,inf,inf,inf
